# Setup

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Exploration

## Connect

In [2]:
URL = 'https://www.boliga.dk/salg/resultater?propertyType=1,2,3&municipality=101&salesDateMin=2015&salesDateMax=today&searchTab=1&sort=date-d&page=1'

# Set user agent
''' Link to find user agent
https://httpbin.org/get
'''
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "Accept-Encoding": "gzip, deflate, br"
}

In [ ]:
text = requests.get(URL,
                    headers = headers
                   )


text

## Read website

In [ ]:
soup1 = BeautifulSoup(text.content, 'html.parser')



print(soup1.prettify())

In [ ]:
soup1.findAll('table')[0].findAll('tr')[0]

In [ ]:
r1 = soup1.findAll('table')[0].findAll('tr')[0]

print(r1)

## Parse website

In [ ]:


r1_c = r1.find_all('td',  class_ = "table-col text-center")

l_values = []
t_address = r1.find('a', class_ = "text-primary font-weight-bolder text-left").text
t_type = r1.find('span', class_= 'icon').text


l_values.append(t_address)
l_values.append(t_type)

for c in r1_c:
    l_values.append(c.text)
    
    
print(l_values)

l_col_names = ['Address','Type', 'Price', 'Date_SellType', 'Size', 'Price_size', 'Rooms', 'Built', 'PriceChange', 'Link']

dict_row = dict(zip(l_col_names, l_values))

print(dict_row)

pd.DataFrame.from_dict([dict_row])

### Loop to parse website

In [ ]:
l_dfs = []

for r in soup1.findAll('table')[0].findAll('tr'):
    
    r1_c = r.find_all('td',  class_ = "table-col text-center")

    l_values = []
    t_address = r1.find('a', class_ = "text-primary font-weight-bolder text-left").text
    t_type = r1.find('span', class_= 'icon').text


    l_values.append(t_address)
    l_values.append(t_type)

    for c in r1_c:
        l_values.append(c.text)


    #print(l_values)

    l_col_names = ['Address','Type', 'Price', 'Date_SellType', 'Size', 'Price_size', 'Rooms', 'Built', 'PriceChange', 'Link']

    dict_row = dict(zip(l_col_names, l_values))

    #print(dict_row)

    df = pd.DataFrame.from_dict([dict_row])
    
    l_dfs.append(df)
    

pd.concat(l_dfs).head()
    
    

## Selenium

In [32]:
wd = webdriver.Chrome(executable_path='chromedriver.exe')

wd.get(URL)
    
wd.find_element_by_xpath('//*[@id="coiPage-1"]/div[2]/div[1]/button[2]').click()



In [35]:
table = wd.find_element_by_xpath('/html/body/app-root/app-scroll-position-restoration/app-main-layout/app-sold-properties-list/div[3]/app-sold-list-table/table')

l_dfs = []

for r in table.find_elements_by_tag_name('tr'):
    
    l_values = []
    
    l_values = r.text.split('\n')

        # Create list with column names
    l_col_names = ['Type', 'StreetAddress','Area', 'Price', 'Date','SellType', 'Size', 'Price_m2', 'Rooms_YearBuilt_PriceChange']

    # Zip values and columns names to dict
    dict_row = dict(zip(l_col_names, l_values))

    # Create data frame from dict
    df = pd.DataFrame.from_dict([dict_row])
    
    l_dfs.append(df)
    
pd.concat(l_dfs)
    
    


,Type,StreetAddress,Area,Price,Date,SellType,Size,Price_m2,Rooms_YearBuilt_PriceChange
0,E,"Amerika Plads 6, 3. th",2100 København Ø,5.000.000 kr.,17-01-2022,Alm. Salg,93 m²,53.763 kr/m²,3 2007 -5% Aktuel værdi
0,E,"Strandvejen 8, 3. tv",2100 København Ø,8.495.000 kr.,14-01-2022,Alm. Salg,170 m²,49.971 kr/m²,6 1929 Aktuel værdi
0,E,"Nørrebrogade 9F, 1. th",2200 København N,5.400.000 kr.,14-01-2022,Alm. Salg,105 m²,51.429 kr/m²,3 1863 -2% Aktuel værdi
0,E,"Badensgade 48, 2",2300 København S,7.200.000 kr.,14-01-2022,Alm. Salg,65 m²,110.769 kr/m²,3 1899 Aktuel værdi
0,E,"Badensgade 48, 1",2300 København S,7.200.000 kr.,14-01-2022,Alm. Salg,79 m²,91.139 kr/m²,3 1899 Aktuel værdi
0,E,"Badensgade 48, st",2300 København S,7.200.000 kr.,14-01-2022,Alm. Salg,79 m²,91.139 kr/m²,3 1899 Aktuel værdi
0,E,"Nattergalevej 52, 1. tv",2400 København NV,2.895.000 kr.,14-01-2022,Alm. Salg,54 m²,53.611 kr/m²,2 1937 0% Aktuel værdi
0,E,"Sorrentovej 14, 1. tv",2300 København S,2.330.000 kr.,14-01-2022,Alm. Salg,52 m²,44.808 kr/m²,2 1940 -2% Aktuel værdi
0,E,"Murmanskgade 3, st. tv",2150 Nordhavn,4.900.000 kr.,14-01-2022,Alm. Salg,162 m²,30.247 kr/m²,6 2017 Aktuel værdi
0,E,"Murmanskgade 3, 3. mf",2150 Nordhavn,8.972.500 kr.,14-01-2022,Alm. Salg,139 m²,64.550 kr/m²,3 2017 Aktuel værdi


In [ ]:
#for r in wd.find_elements_by_tag_name('tr'):
#    print(r.text)
l_table_rows = WebDriverWait(wd, 10).until(
EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
)

for r in l_table_rows:
    print(r.text)

,Type,StreetAddress,Area,Price,Date,SellType,Size,Price_m2,Rooms_YearBuilt_PriceChange
0,E,"Amerika Plads 6, 3. th",2100 København Ø,5.000.000 kr.,17-01-2022,Alm. Salg,93 m²,53.763 kr/m²,3 2007 -5% Aktuel værdi


In [46]:
buttons = wd.find_elements_by_class_name('page-button')

l_n_pages = []
for button in buttons:
    try:
        n_pages = int(button.text)
        print(n_pages)
        
        l_n_pages.append(n_pages)
    
    except:
        next

max(l_n_pages)

1
8
9
10
11
12
1078
1
8
9
10
11
12
1078


1078

In [44]:
i = 1
while i < 10:
    print(i)
    i += 1
    wd.find_element_by_xpath('/html/body/app-root/app-scroll-position-restoration/app-main-layout/app-sold-properties-list/div[3]/div/div/app-pagination/div/div[4]/a').click()

1
2
3
4
5
6
7
8
9


In [ ]:
l_n_pages = []
for button in soup1.find_all('a', class_= 'page-button'):
    try:
        n_pages = int(button.text)
        print(n_pages)
        
        l_n_pages.append(n_pages)
    
    except:
        next

max(l_n_pages)

In [ ]:
wd.

# Program

## Connect

In [2]:
URL = 'https://www.boliga.dk/salg/resultater?propertyType=1,2,3&municipality=101&salesDateMin=2015&salesDateMax=today&searchTab=1&sort=date-d&page=1'

# Set user agent
''' Link to find user agent
https://httpbin.org/get
'''
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36',
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
    "Accept-Encoding": "gzip, deflate, br"
}

## Function

### Function 1
Create function to scrape single page from Boliga.

In [51]:
def fun_scrape_boliga(
    wd, # Webdriver session
    #URL, # link to search
    do_print  = 0 # set 1 to print
):
    
    # Find table element
    
    table = WebDriverWait(wd, 10).until(
    EC.presence_of_element_located((By.XPATH, '/html/body/app-root/app-scroll-position-restoration/app-main-layout/app-sold-properties-list/div[3]/app-sold-list-table/table'))
    )

    #table = wd.find_element_by_xpath('/html/body/app-root/app-scroll-position-restoration/app-main-layout/app-sold-properties-list/div[3]/app-sold-list-table/table')
    
    #print(table.text)
    # Initialize empty list for data frames
    l_dfs = []
    
    
    
    l_table_rows = WebDriverWait(wd, 20).until(
    EC.presence_of_all_elements_located((By.TAG_NAME, 'tr'))
    )
    
    
    
    # Loop through table rows
    for r in l_table_rows:
        
        #print(r.text)
        # Split text to list
        
        l_values = r.text.split('\n')
        
            
        #print(l_values)

        # Create list with column names
        l_col_names = ['Type', 'StreetAddress','Area', 'Price', 'Date','SellType', 'Size', 'Price_m2', 'Rooms_YearBuilt_PriceChange']

        # Zip values and columns names to dict
        dict_row = dict(zip(l_col_names, l_values))

        # Create data frame from dict
        df = pd.DataFrame.from_dict([dict_row])

        l_dfs.append(df)
        
        if print == 1:
            print('Row {} parsed'.format(count))
    
        
    # Concatenate dataframes to one, and return
    df_out = pd.concat(l_dfs)
    
    print('Page parsed')
    return df_out  


### Function 2
Create function to loop through all pages of search on Boliga

In [61]:
def fun_loop_boliga(
    wd, # Webdriver session
   # URL, # link to Boliga search
    max_pages = 2000,
    sleep_time = 2
):

    ## Find number of pages ## 
    # Find buttons 
    l_buttons = wd.find_elements_by_class_name('page-button')
    
    # Initialize empty list to store number of pages
    l_n_pages = []
    
    # For ech bottong, save text in list
    for button in l_buttons:
        try:
            n_pages = int(button.text)
            
            l_n_pages.append(n_pages)
        except:
            next
            
    # Find maximum in list
    max(l_n_pages)
    n_pages = max(l_n_pages)
    print('Number of pages: {}'.format(n_pages))
      
    # Change number of pages if maximum is exceeded
    if max_pages < n_pages:
        n_pages = max_pages
        print('Number of pages set to max. pages: {}'.format(n_pages))
    
    
    ## Loop through pages ## 
    # Initialize empty list of data frames
    l_dfs = []
    
    i = 1
    while i <= n_pages:

        time.sleep(sleep_time)
        
        # Scrape page
        df = fun_scrape_boliga(wd)
        
        df.to_csv('Files/BoligaScraping_Page_{}.csv'.format(i))
        
        print(i)
        i += 1
                
        # Append data frame to list
        l_dfs.append(df)
        
        #print('Page {} scraped'.format(i))
        wd.find_element_by_xpath('/html/body/app-root/app-scroll-position-restoration/app-main-layout/app-sold-properties-list/div[3]/div/div/app-pagination/div/div[4]/a').click()
        
        
    # Concatenate data frames
    df_out = pd.concat(l_dfs)
    
    return(df_out)
                               
                               

In [59]:
wd = webdriver.Chrome(executable_path='chromedriver.exe')

wd.get(URL)


    
wd.find_element_by_xpath('//*[@id="coiPage-1"]/div[2]/div[1]/button[2]').click()


df = fun_loop_boliga(wd,max_pages = 1100, sleep_time = 3)


Number of pages: 1078
Page parsed
1
Page parsed
2
Page parsed
3
Page parsed
4
Page parsed
5
Page parsed
6
Page parsed
7
Page parsed
8
Page parsed
9
Page parsed
10
Page parsed
11
Page parsed
12
Page parsed
13
Page parsed
14
Page parsed
15
Page parsed
16
Page parsed
17
Page parsed
18
Page parsed
19
Page parsed
20
Page parsed
21
Page parsed
22
Page parsed
23
Page parsed
24
Page parsed
25
Page parsed
26
Page parsed
27
Page parsed
28
Page parsed
29
Page parsed
30
Page parsed
31
Page parsed
32
Page parsed
33
Page parsed
34
Page parsed
35
Page parsed
36
Page parsed
37
Page parsed
38
Page parsed
39
Page parsed
40
Page parsed
41
Page parsed
42
Page parsed
43
Page parsed
44
Page parsed
45
Page parsed
46
Page parsed
47
Page parsed
48
Page parsed
49
Page parsed
50
Page parsed
51
Page parsed
52
Page parsed
53
Page parsed
54
Page parsed
55
Page parsed
56
Page parsed
57
Page parsed
58
Page parsed
59
Page parsed
60
Page parsed
61
Page parsed
62
Page parsed
63
Page parsed
64
Page parsed
65
Page parsed


In [62]:
print(df_test.shape)

df_test.to_csv('Data_Boliga_20220208.csv', index  = False)

(54793, 9)


In [91]:
df_test

,Type,StreetAddress,Area,Price,Date,SellType,Size,Price_m2,Rooms_YearBuilt_PriceChange
0,E,"Amerika Plads 6, 3. th",2100 København Ø,5.000.000 kr.,17-01-2022,Alm. Salg,93 m²,53.763 kr/m²,3 2007 -5% Aktuel værdi
0,E,"Strandvejen 8, 3. tv",2100 København Ø,8.495.000 kr.,14-01-2022,Alm. Salg,170 m²,49.971 kr/m²,6 1929 Aktuel værdi
0,E,"Nørrebrogade 9F, 1. th",2200 København N,5.400.000 kr.,14-01-2022,Alm. Salg,105 m²,51.429 kr/m²,3 1863 -2% Aktuel værdi
0,E,"Badensgade 48, 2",2300 København S,7.200.000 kr.,14-01-2022,Alm. Salg,65 m²,110.769 kr/m²,3 1899 Aktuel værdi
0,E,"Badensgade 48, 1",2300 København S,7.200.000 kr.,14-01-2022,Alm. Salg,79 m²,91.139 kr/m²,3 1899 Aktuel værdi
...,...,...,...,...,...,...,...,...,...
0,E,"Esromgade 26, 2",2200 København N,8.350.000 kr.,02-01-2015,Alm. Salg,106 m²,78.774 kr/m²,2 1904 Aktuel værdi
0,E,"Esromgade 26, 1. tv",2200 København N,8.350.000 kr.,02-01-2015,Alm. Salg,56 m²,149.107 kr/m²,1 1904 Aktuel værdi
0,E,"Esromgade 26, 3. th",2200 København N,8.350.000 kr.,02-01-2015,Alm. Salg,59 m²,141.525 kr/m²,1 1904 Aktuel værdi
0,E,"Esromgade 26, 4",2200 København N,8.350.000 kr.,02-01-2015,Alm. Salg,183 m²,45.628 kr/m²,6 1904 Aktuel værdi


# Clean data

In [2]:
df_test =  pd.read_csv('Data_Boliga_20220208.csv')

df = df_test.copy()

df



,Type,StreetAddress,Area,Price,Date,SellType,Size,Price_m2,Rooms_YearBuilt_PriceChange
0,E,"Amerika Plads 6, 3. th",2100 København Ø,5.000.000 kr.,17-01-2022,Alm. Salg,93 m²,53.763 kr/m²,3 2007 -5% Aktuel værdi
1,E,"Strandvejen 8, 3. tv",2100 København Ø,8.495.000 kr.,14-01-2022,Alm. Salg,170 m²,49.971 kr/m²,6 1929 Aktuel værdi
2,E,"Nørrebrogade 9F, 1. th",2200 København N,5.400.000 kr.,14-01-2022,Alm. Salg,105 m²,51.429 kr/m²,3 1863 -2% Aktuel værdi
3,E,"Badensgade 48, 2",2300 København S,7.200.000 kr.,14-01-2022,Alm. Salg,65 m²,110.769 kr/m²,3 1899 Aktuel værdi
4,E,"Badensgade 48, 1",2300 København S,7.200.000 kr.,14-01-2022,Alm. Salg,79 m²,91.139 kr/m²,3 1899 Aktuel værdi
...,...,...,...,...,...,...,...,...,...
54788,E,"Esromgade 26, 2",2200 København N,8.350.000 kr.,02-01-2015,Alm. Salg,106 m²,78.774 kr/m²,2 1904 Aktuel værdi
54789,E,"Esromgade 26, 1. tv",2200 København N,8.350.000 kr.,02-01-2015,Alm. Salg,56 m²,149.107 kr/m²,1 1904 Aktuel værdi
54790,E,"Esromgade 26, 3. th",2200 København N,8.350.000 kr.,02-01-2015,Alm. Salg,59 m²,141.525 kr/m²,1 1904 Aktuel værdi
54791,E,"Esromgade 26, 4",2200 København N,8.350.000 kr.,02-01-2015,Alm. Salg,183 m²,45.628 kr/m²,6 1904 Aktuel værdi


Clean text data

In [3]:
# Clean text columns
df['Price'] = df['Price'].str.replace('.', '', regex = True).str.replace('kr', '', regex = True)
df['Size'] = df['Size'].str.extract('(\d+)')
df['Price_m2']  = df['Price_m2'].str.replace('.', '', regex = True).str.extract('(\d+)')
df[['Rooms', 'YearBuilt', 'PriceChange']] =  df['Rooms_YearBuilt_PriceChange'].str.split(' ', expand = True).iloc[:,0:3]
df.loc[df['PriceChange'] == 'Aktuel', ['PriceChange']] = '0'
df['PriceChange'] = df['PriceChange'].str.replace('%', '', regex  =True)

df['PostalCode'] = df['Area'].str.extract('(\d{4})')
df['AreaName'] = df['Area'].str.extract('(\D+)')

df.head()


,Type,StreetAddress,Area,Price,Date,SellType,Size,Price_m2,Rooms_YearBuilt_PriceChange,Rooms,YearBuilt,PriceChange,PostalCode,AreaName
0,E,"Amerika Plads 6, 3. th",2100 København Ø,5000000,17-01-2022,Alm. Salg,93,53763,3 2007 -5% Aktuel værdi,3,2007,-5,2100,København Ø
1,E,"Strandvejen 8, 3. tv",2100 København Ø,8495000,14-01-2022,Alm. Salg,170,49971,6 1929 Aktuel værdi,6,1929,0,2100,København Ø
2,E,"Nørrebrogade 9F, 1. th",2200 København N,5400000,14-01-2022,Alm. Salg,105,51429,3 1863 -2% Aktuel værdi,3,1863,-2,2200,København N
3,E,"Badensgade 48, 2",2300 København S,7200000,14-01-2022,Alm. Salg,65,110769,3 1899 Aktuel værdi,3,1899,0,2300,København S
4,E,"Badensgade 48, 1",2300 København S,7200000,14-01-2022,Alm. Salg,79,91139,3 1899 Aktuel værdi,3,1899,0,2300,København S


Modify values throwing errors

In [4]:
df[df.isnull().any(axis = 1)]
df.loc[df['Price_m2'].isnull(), 'Price_m2'] = '0'


df[df.isin(['299.499.900']).any(axis = 1)]
df.loc[df['PriceChange'] == '299.499.900', 'PriceChange'] = '0'

Drop redundant column

In [5]:
# Drop redundant column
df.drop('Rooms_YearBuilt_PriceChange', inplace = True, axis = 1)

Write clean data

In [6]:
df.to_csv('Data_Boliga_20220208_cleaned.csv', index  = False)

In [128]:


# Convert columns to integer
l_cols_int = ['Price', 'Size', 'Price_m2','YearBuilt', 'PriceChange']
df[l_cols_int] = df[l_cols_int].astype(int)

# Convert datetime column
df['Date'] = pd.to_datetime(df['Date'])

# Convert to categorical columns
l_cols_cat = ['Type', 'SellType', 'Rooms', 'PostalCode']
df[l_cols_cat] = df[l_cols_cat].astype('category')

df.head()


,Type,StreetAddress,Area,Price,Date,SellType,Size,Price_m2,Rooms,YearBuilt,PriceChange,PostalCode,AreaName
0,E,"Amerika Plads 6, 3. th",2100 København Ø,5000000,2022-01-17,Alm. Salg,93,53763,3,2007,-5,2100,København Ø
0,E,"Strandvejen 8, 3. tv",2100 København Ø,8495000,2022-01-14,Alm. Salg,170,49971,6,1929,0,2100,København Ø
0,E,"Nørrebrogade 9F, 1. th",2200 København N,5400000,2022-01-14,Alm. Salg,105,51429,3,1863,-2,2200,København N
0,E,"Badensgade 48, 2",2300 København S,7200000,2022-01-14,Alm. Salg,65,110769,3,1899,0,2300,København S
0,E,"Badensgade 48, 1",2300 København S,7200000,2022-01-14,Alm. Salg,79,91139,3,1899,0,2300,København S


In [132]:
df_in = pd.read_csv('Data_Boliga_20220208_cleaned.csv')

df_in.dtypes

Type             object
StreetAddress    object
Area             object
Price             int64
Date             object
SellType         object
Size              int64
Price_m2          int64
Rooms             int64
YearBuilt         int64
PriceChange       int64
PostalCode        int64
AreaName         object
dtype: object

# Old

In [ ]:
# Function
def fun_scrape_boliga(
    URL, # link to search
    do_print  = 0 # set 1 to print
):
    # Request page
    text = requests.get(URL,
                    headers = headers
                   )
    
    # Create soup of content
    soup1 = BeautifulSoup(text.content, 'html.parser')
    
    ## Parse website ##
    # Initialize empty list for data frames
    l_dfs = []
    
    # Loop through rows
    for count, r in enumerate(soup1.findAll('table')[0].findAll('tr')):
        
        # Find data for row
        r_c = r.find_all('td',  class_ = "table-col text-center")
        
        # Initialize empty list for values
        l_values = []
        
        ## Extract values ##
        # Address
        t_address = r.find('a', class_ = "text-primary font-weight-bolder text-left").text
        # House type
        t_type = r.find('span', class_= 'icon').text
        
        # Appned values
        l_values.append(t_address)
        l_values.append(t_type)
        
        
        # Loop through items in data
        for c in r_c:
            # Append values to list
            l_values.append(c.text)
            
            
        # Create list with column names
        l_col_names = ['Address','Type', 'Price', 'Date_SellType', 'Size', 'Price_size', 'Rooms', 'Built', 'PriceChange', 'Link']
        
        # Zip values and columns names to dict
        dict_row = dict(zip(l_col_names, l_values))
        
        # Create data frame from dict
        df = pd.DataFrame.from_dict([dict_row])
        
        # Append data frame to list of data frames
        l_dfs.append(df)
        
        if print == 1:
            print('Row {} parsed'.format(count))
        
    # Concatenate dataframes to one, and return
    df_out = pd.concat(l_dfs)
    
    print('Page parsed')
    return df_out  


In [83]:
2000/60

33.333333333333336